앞서 \ref{chap:TyInf}장에서 TODO
\lipsum[3-3]

TODO 일단 12장 내용을 템플릿으로 복사만 해 놓은 상태 TODO

In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

\newpage
# 재귀함수요약식의 타입복원이 어려운 이유
\label{sec:WhyRecTyReconHard}
앞서 \ref{sec:recTyRecon}절에서 $\textbf{rec}\,f\,(\lambda x{:}\tau_2.e)$와 같은 재귀함수요약식에서 $f$의
타입 표기 복원이 어렵다고 언급한 바 있는데 여기서는 그 이유를 생각해 보자.
타입복원(`recon`) 함수는 타입환경(`TEnv`)과 식(`Expr`)을 인자로 받아 
타입유추(`infer`) 함수를 활용해 부분식으로부터 복원에 필요한 타입 정보를 얻어가며
생략된 타입 표기에 대한 복원을 진행한다.
설명의 편의를 위해 하스켈 코드와 타입규칙의 표현을 혼용한 의사코드(pseudocode)로 나타내자면
대략 다음과 같이 진행되어야 할텐데
\newline
`recon` $\Gamma$ $(\textbf{rec}\,f\,(\lambda x{:}\tau_2.e))$ = `[` $\!\!\textbf{rec}\,f{:}\tau_1\,(\lambda x{:}\tau_2.e)\!$ `|` $\!\tau_1$ $\leftarrow$ `infer` $\{x{:}\tau_2,\,f{:}??\}\Gamma$ $e\!\!$ `]` 
\newline
문제는 $??$의 위치가 과연 무슨 타입이어야 하는가이다.
$f$는 재귀함수요약식 자체를 대표하므로 당연히 $\tau_1$이면 되지 않냐고 생각했다면,
물론 맞는 생각이기는 하지만, 문제는 그런 방식으로 프로그램을 작성할 수가 없다는 것이다.
왜냐하면 $\tau_1$은 타입유추(`infer`)의 계산 결과로 비로소 알게 되는 타입인데,
`infer` 함수의 첫째 인자인 확장된 환경이 $\tau_1$을 포함한다면 이는 계산하기도 전에
결과값을 미리 알고 제공해야 한다는 이야기이므로 말이 되지 않는다.

이런 어려움을 극복하려면 말이 되지 않는 상황을 회피할 방법을 찾으면 된다.
말이 되지 않았던 이유는 $\tau_1$이라는 계산 결과를 미리 \`완전히\' 알고 타입환경에 $f$의 $\tau_1$에 대한 바인딩을 포함해야 한다는 생각 때문이었다.
그렇다면 불완전한 타입 정보를 타입 환경의 바인딩 내용으로 제공하거나 심지어 식에도 불완전한 타입 정보를 표기하는 것을 허용한다면 어떨까?
불완전한 정보 중에서 알려지지 않은 부분이 결국 무엇인지는 여전히 해결해야 할 숙제로 미뤄지긴 하겠지만, 최소한 완전히 말이 되지 않는 상황에서는 일단 벗어나서 생각해 볼 수 있다.
사실 이러한 접근 방식은 우리가 이미 수학에서 미지수를 나타내는 변수를 포함한 \`방정식\'의 풀이에서 익숙한 문제 해결 방식이다.
예컨대, $x = 2\times x - 3$이라는 방정식은 미지수 $x$를 포함한 우변의 산술식 $2\times x - 3$의 계산 결과가 좌변, 즉 $x$라는 말이다.
이렇게 아직 알려지지 않은 계산 결과를 일부분으로 포함하는 방정식을 해결하는 방법이 있듯, 재귀함수요약식의 타입복원도 마찬가지로 해결할 수 있으리라는 희망을 가져 볼 수 있다.

# 미지의 타입 정보를 나타내는 타입변수
\label{sec:UnkownTV}
미지의 타입 정보를 나타내는 타입변수를 FACT언어의 타입을 나타내는 데이터 타입 `Ty`에 다음과 같이 추가하였다.
타입변수는 하스켈 프로그램에서 `TV i`로 작성하며 (여기서 `i::Int`) 보기좋게 노트북에 디스플레이할 때는 $t_i$로 표시되도록 하였다.
앞서 다룬 예시인 $\textbf{rec}\,f\,(\lambda x{:}\mathbb{Z}.e)$의 타입을 복원하려 할 때 타입변수를 활용해 생각을 한번 펼쳐 보자.
전체 재귀함수를 대표하는 $f$는 정수($\mathbb{Z}$) 타입의 인자를 받는다는 것은 알지만 결과값의 타입이 무엇이어야 하는지는 아직 잘 모른다.
따라서 $f$의 타입을 $\mathbb{Z}{\to}t_i$(하스켈 코드로는 `Arr Z (TV i)`)로 바인딩해
확장된 타입환경 $\{f{:}\mathbb{Z}{\to}t_i\}\Gamma$에서 $e$의 타입으로 유추된 결과 $\tau_1$과
$t_i$를 일치화(unify)하는 방정식을 구성하여 그 해를 구할 수 있다면 알맞은 타입을 찾게 된 것이다.
그러나, 방정식을 구성했더라도 해가 존재하지 않는다고 판단된다면 타입규칙에 맞는 타입이 없다는 말이다.

In [2]:
type Nm = String  -- 변수 이름은 문자열로
data Ty = TV Int | Z | Arr Ty Ty  deriving (Eq,Ord)
data Expr = Var Nm                      -- x
          | Lam Nm (Maybe Ty) Expr      -- (\x:t.e)
          | Rec Nm (Maybe Ty) Expr      -- rec f:t (\x:t2.e)
          | App Expr Expr               -- (e1 e2)
          | Lit Int                     -- n
          | Add Expr Expr               -- e1 + e2
          | If Expr Expr Expr           -- if e then e1 else e0
          deriving (Eq, Ord)
data Val = Cl Expr Env | Vi Int  deriving (Eq,Ord)
type Env = [ (Nm, Val) ]
type TEnv = [ (Nm, Ty) ]

In [3]:
instance Show Ty where
  showsPrec p (TV i) = showString("t_{"++show i++"}")
  showsPrec _ Z = showString "\\mathbb{Z}"
  showsPrec p (Arr t1 t2) = showParen (p > 1) $
      showsPrec p t1 . showString "{\\to}" . showsPrec p t2
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x mt e) = showParen (p > 1) $
      showString ("\\lambda "++x) . showsMT mt .
      showString "." . showsPrec 1 e
  showsPrec p (Rec f mt e) = showParen (p > 9) $
      showString ("\\textbf{rec}\\,"++f) . showsMT mt .
      showString "\\, " . showsPrec 2 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
  showsPrec _ (Lit n) = shows n
  showsPrec p (Add e1 e2) = showParen (p > 6) $
      showsPrec 6 e1 . showString "+" . showsPrec 7 e2
  showsPrec p (If e e1 e0) = showParen (p > 2) $
      showString "\\textbf{if}\\;" . showsPrec 3 e .
      showString "\\;\\textbf{then}\\;" . showsPrec 3 e1 .
      showString "\\;\\textbf{else}\\;" . showsPrec 3 e0

showsMT (Just t) = showString "{:}" . showsPrec 0 t
showsMT Nothing  = id 

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Ty where
    display t = display [latex $ "$"++show t++"$"]

In [4]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show TEnv where
    show tenv = "\\{"++ intercalate ",\\," (map show tenv) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Val) where
    show (x,v) = x++"{\\mapsto}"++show v
instance {-# OVERLAPS #-} Show (Nm,Ty) where
    show (x,t) = x++"{\\mapsto}"++show t
instance Show Val where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "
    show (Vi n) = " "++show n++" "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay TEnv where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Val) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Ty) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Val where
    display v = display [latex $ "$"++show v++"$"]

# 다소 무책임한 타입복원(`tyrecon`)
지금까지는 기존의 FACT언어에 대한 타입복원과 타입유추에 대한 이해를 바탕으로 설명하기 위해
\ref{chap:TyInf}장에서 타입복원(`tyrecon`) 함수에 타입유추(`tyinf`) 함수를 활용하던 흐름에 맞춰
왜 재귀함수타입요약식의 타입복원 및 타입유추가 어려운지 이야기하였다 (\ref{sec:WhyRecTyReconHard}절).
그런데, \ref{sec:UnknownTV}절에서 언급한 바와 같이 결국 타입복원 및 타입유추가 최종적으로 성공했는지 실패했는지는,
그 과정에서 구성된 방정식의 해를 찾을 수 있는지 없는지에 대한 판단을 포함한 방정식의 풀이에 달려 있다.
그러니까 타입복원 과정에서 일을 많이 하더라도 조금 더 풀기 쉬운 형태의 방정식을 구성할 수는 있을지언정
일반적으로 알맞은 타입이 있는지 결론내지는 못한다. 그래서, 타입유추 등 다른 부분에서 원활한 진행을 위한
최소한의 작업만 하도록, 어찌 보면 조금 무책임하게 방정식을 푸는 쪽에 할 일을 몰아서 미루는 방식으로,
`recon`을 작성하자. 미지의 타입 정보를 타입변수로 나타낼 수 있다는 점에 최대한 활용해 타입이 생략되어
`Nothing`으로 나타난 곳에 서로 겹치지 않는 고유한 타입변수 `Just (TV i)`로 그냥 복원해 버리자는 것이다.
원리는 이렇게 간단한데, 서로 겹치지 않는 고유한 정수 `i`를 각각의 `Nothing`의 위치에 배치되도록 구현하기
위한 내용으로 인해 아래 코드가 조금 길어졌다. 이렇게 되면, `recon`이 무조건 성공하므로 결과 타입을
더 이상 실패한 경우도 고려하는 `[Expr]`로 할 필요 없이 그냥 `Expr`로 바꿀 수 있다.

In [5]:
recon :: TEnv -> Expr -> Expr  -- 각각의 Nothing을 고유한 타입변수로
recon tenv e = rcn 0 tenv e

rcn i _    e@(Var _)    = e
rcn i tenv (Lam x Nothing  e) = Lam x (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (x,t):(tenv)
                                  t = TV i
rcn i tenv (Lam x (Just t) e) = Lam x (Just t) (rcn i tenv' e)
                            where tenv' = (x,t):tenv
rcn i tenv (Rec f Nothing  e) = Rec f (Just t) (rcn (1+1) tenv' e)
                            where tenv' = (f,t):(tenv)
                                  t = TV i
rcn i tenv (Rec f (Just t) e) = Rec f (Just t) (rcn (1+i) tenv' e)
                            where tenv' = (f,t):tenv
rcn i tenv (App e1 e2)  = App e1' e2'   where e1' = rcn i tenv e1
                                              e2' = rcn j tenv e2
                                              j = 1 + maxTV i e1'
rcn i _    e@(Lit _)    = e
rcn i tenv (Add e1 e2)  = Add e1' e2'   where e1' = rcn i tenv e1
                                              e2' = rcn j tenv e2
                                              j = 1 + maxTV i e1'
rcn i tenv (If e e1 e0) = If e' e1' e0' where e'  = rcn i tenv e
                                              e1' = rcn j tenv e1
                                              e0' = rcn k tenv e0
                                              j = 1 + maxTV i e'
                                              k = 1 + maxTV j e1'

-- maxTV i e는 i를 포함해 e에 나타나는 모든 타입변수 정수값들의 최대값 
maxTV :: Int -> Expr -> Int
maxTV i e = maximum (i : concatMap collectTV (collectTy e))

collectTy :: Expr -> [Ty]  -- 식에 나타나는 모든 타입 표기로 이루어진 리스트
collectTy (Var _)      = []
collectTy (Lam x Nothing  e) = collectTy e
collectTy (Lam x (Just t) e) = t : collectTy e
collectTy (Rec x Nothing  e) = collectTy e
collectTy (Rec x (Just t) e) = t : collectTy e
collectTy (App e1 e2)  = collectTy e1 ++ collectTy e2
collectTy (Lit _)      = []
collectTy (Add e1 e2)  = collectTy e1 ++ collectTy e2
collectTy (If e e1 e0) = collectTy e ++ collectTy e1 ++ collectTy e0

collectTV :: Ty -> [Int]  -- 타입에 나타나는 모든 타입변수 정수값들의 리스트
collectTV (TV i)      = [i]
collectTV Z           = []
collectTV (Arr t1 t2) = collectTV t1 ++ collectTV t2

In [6]:
e1 = Lam "x" Nothing . Lam "x'" Nothing $ Var "x" `Add` Var "x'"  
e2 = Lit 2 `Add` Var "y"  -- y는 자유변수!!!
e5 = Lam "z" Nothing (Var "z") `App` Lit 5
App (App e1 e2) e5

\noindent
위 식에서는 하스켈 코드에서 `Nothing`으로 표기된 세 군데에 타입 표기가 생략되어 있다.
이를 아래와 같이 이 장에서 새로 작성한 `recon` 함수로 타입복원하면 각각의 위치에
겹치지 않는 타입변수 $t_0$, $t_1$, $t_2$가 표기된 식이 만들어진다.

In [7]:
recon [("y",Z)] (App (App e1 e2) e5)

\noindent
참고로 타입환경(`[("y",Z)]`)는 이 장에서 새로 작성한 `recon`에서 활용하지 않으므로 (다른 추가 기능을 구현하지 않는 한)
사실상 필요없는 인자이지만 \ref{chap:TyInf}장과 비슷한 형식으로 일관성을 유지하기 위해 남겨놓았다.

# 풀어야 할 연립방정식을 함께 만들어내는 타입유추 과정
이제 타입유추 함수(`infer`)는 타입(`Ty`)과 함께 풀어야 할 방정식(`Eqn`)도 함께 만들어내야 한다.
어차피 방정식의 해가 존재하는가의 여부에 따라 알맞은 타입의 존재 여부가 결정되므로,
앞서 `recon`을 항상 결과값으로 식(`Expr`)을 만들어내는 데 성공하는 것과 마찬가지 논리로
`infer`도 항상 결과값으로 타입(`Ty`)을 만들어내는 데 거의 항상 성공하는 것으로 취급할 수 있다.
단, 이름의 유효범위가 잘못되었거나 타입복원을 제대로 수행하지 않은 식에 대해서는
방정식을 굳이 풀어 볼 필요도 없이 너무나 명확히 실패하는 경우이므로  `error` 함수로 비정상 종료 처리한다.
따라서 기존에 실패하는 경우를 고려한 `[Ty]` 대신 그냥 `Ty`를 결과값으로 계산하면 되는데,
이와 함께 연립방정식(즉, 여러 개의 방정식으로 이루어진 리스트 `[Eqn]`)도 만들어내야 하므로,
`infer`의 결과값은 이 두 정보의 순서쌍 타입인 `(Ty, [Eqn])`으로 표현된다.

In [8]:
type Eqn = (Ty,Ty) -- 방정식의 의미는 두 타입이 일치화(unify)될 수 있는지

infer :: Int -> TEnv -> Expr -> (Ty,[Eqn])
infer i tenv (Var x) =
  case lookup x tenv of
    Just t  -> (t, [])
    Nothing -> error( x ++ " not found in " ++ show tenv )
infer i tenv (Lam x (Just t2) e) = (Arr t2 t, eqns)
  where (t,eqns) = infer i ((x,t2):tenv) e
infer i tenv (Lam x Nothing   e) =
  error( "missing type annot. in " ++ show(Lam x Nothing e) )
infer i tenv (Rec f (Just t) e)  = undefined
infer i tenv (Rec f Nothing  e)  =
  error( "missing type annot. in " ++ show(Rec f Nothing e) )
infer i tenv (App e1 e2)  = undefined
infer i tevv (Lit _)      = (Z, [])
infer i tenv (Add e1 e2)  = (Z, eqns1 ++ eqns2)
  where eqns1 = check i tenv e1 Z
        eqns2 = check j tenv e2 Z
        j = 1 + maximum (maxTV i e1 : collectTVeqns eqns1)
infer i tenv (If e e1 e0) = undefined

collectTVeqns eqns = [i | (t',t) <- eqns,
                          i <- collectTV t' ++ collectTV t]

check :: Int -> TEnv -> Expr -> Ty -> [Eqn]
check i tenv e t = (t',t) : eqns  where (t',eqns) = infer i tenv e

\noindent
타입검사 함수(`check`)를 타입유추 함수(`infer`)로 간단히 정의하는 것은 마찬가지지만,
당장 유추된 타입(`t'`)과 제시된 타입(`t`)이 같은가 판단하기에는 정보가 불완전할지 모르므로 일단 보류하고,
두 타입이 일치화 가능함을 의미하는 새로운 방정식을 구성해 타입유추로부터 얻은 연립방정식(`eqns`)에 추가한다.
따라서, `check`의 결과값은 더 이상 `Bool` 타입이 아닌 `[Eqn]` 타입이 된다.
이 장의 `infer`와 `check`에 추가된 파라메터 `i`는
`[6]`번 셀에 선언된 `rcn` 함수의 파라메터 `i`와 마찬가지로
새로운 타입변수의 정수값을 기존의 타입변수와 겹치지 않도록
어디서부터 시작할지 알려주는 정수 인자를 전달하기 위한 파라메터이다.

In [9]:
instance {-# OVERLAPS #-} Show [Eqn] where
    show eqns = "\\{"++ intercalate ",\\," (map show eqns) ++"\\}"
instance {-# OVERLAPS #-} Show Eqn where
    show (t',t)= show t'++"\\,\\overset{?}{=}\\,"++show t

instance {-# OVERLAPS #-} IHaskellDisplay Eqn where
    display eqn = display [latex $ "$"++show eqn++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay [Eqn] where
    display eqns = display [latex $ "$"++show eqns++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Ty,[Eqn]) where
    display e = display [latex $ "$"++show e++"$"]

In [10]:
Var "x" `Add` Var "x'" -- e1의 함수 몸체 부분에 대한 예시
infer 2 [("x'",TV 1),("x",TV 0)] (Var "x" `Add` Var "x'")

위의 연립방정식에 만족하는 해($\{t_0{\mapsto}\mathbb{Z},t_1{\mapsto}\mathbb{Z}\}$)가 존재하므로
$x+x'$에 대해 주어진 타입환경에서 타입규칙에 맞는 타입 $\mathbb{Z}$을 찾을 수 있다고 판단한다.
하지만 아래와 같은 경우처럼, 똑같은 식이라도 주어지는 타입환경에 따라서는 연립방정식을 만족하는 해가 존재하지 않는다.
연립방정식을 만족한다는 것은 연립방정식에 속한 각각의 방정식을 모두 참으로 만들 수 있어야 한다는 말인데,
문제는 $\mathbb{Z}{\to}\mathbb{Z}\,\overset{?}{=}\,\mathbb{Z}$는 애초부터 만족 불가능한 방정식이기에
이 방정식이 포함된 연립방적식은 해가 존재할 수 없다. 이와 같이 에러 없이 실행되어 순서쌍의 첫 요소로 
구체적인 타입이 계산되더라도 연립방정식에 따라 타입유추의 최종적인 성패가 갈리게 됨을 유의해야 한다.

In [11]:
Var "x" `Add` Var "x'" -- 잘못된 타입환경이 주어진 경우의 예시
infer 2 [("x'",Arr Z Z),("x",TV 0)] (Var "x" `Add` Var "x'")

In [12]:
e1  -- e1은 타입 표기가 생략된 곳이 있다
infer 0 [] e1 -- 그래서 infer는 에러남

: 

In [13]:
e1' = recon [] e1 
e1'         -- e1의 타입을 복원한 e1'

In [14]:
maxTV 0 e1' -- e1'의 최대 타입변수 정수값

1

In [15]:
infer (maxTV 0 e1' + 1) [] e1' -- 타입을 복원하여 infer하면 에러없이 실행됨

In [16]:
infer' e = infer (maxTV 0 e + 1) [] e -- 닫힌식 e에 대한 타입유추 함수

\noindent
아직 `[8]`번 셀에서 `undefined`로 남겨져 있는 부분이 있기 때문에 일반적으로 `Var`, `Lam`, `Add` 외의 다른 문법요소가 포함된 식에 대해서는 타입유추(`infer`)가 제대로 동작하지 않는다.
연습문제로 `[8]`번 셀의 `infer` 함수를 완성하여 다른 문법요소가 포함된 식에 대한 타입유추도 `infer` 또는 `infer'`로 테스트해 보라.

# 일치화 방정식의 풀이
\label{sec:SolveUnify}
\ref{sec:CESK}절에서 CESK기계를 작성하며 활용한 `HashMap` 데이터 구조로 일치화(unification) 방정식의 해인 타입 치환을 나타내기로 하자.

\url{https://www.cs.bu.edu/fac/snyder/publications/UnifChapter.pdf}

In [17]:
import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이

type TSubs = HashMap Int Ty -- 타입변수의 정수값을 타입에 대응시키는 타입 치환

emptyTSubs = M.empty :: TSubs -- 빈 타입 치환

In [18]:
solve :: [Eqn] -> [TSubs] -- 해가 없으면 빈 리스트
solve eqns = sol eqns emptyTSubs

sol :: [Eqn] -> TSubs -> [TSubs]
sol []                    tsubs = [tsubs] -- 더 처리할 방정식 없음
sol ((Z, Z)       : eqns) tsubs = sol eqns tsubs -- 항등식 제거
sol ((Arr t1 t2,  -- 방정식 분해 (함수의 정의역끼리 공역끼리)
      Arr s1 s2)  : eqns) tsubs = sol ((t1,s1):(t2,s2):eqns) tsubs
sol ((Z, Arr _ _) : eqns) tsubs = [] -- 정수와 함수의 구조적 불일치
sol ((Arr _ _, Z) : eqns) tsubs = [] -- 정수와 함수의 구조적 불일치
sol ((TV i, TV j) : eqns) tsubs
    | i >  j = sol ((TV j,TV i):eqns) tsubs -- 좌우반전
    | i == j = sol eqns tsubs -- 항등식 제거
sol ((TV i, t) : eqns) tsubs
    | occurs i t' = []  -- 부분이 전체를 포함하므로 해결 불가능
    | otherwise   = sol eqns (insertTSubs i t' tsubs) -- 핵심 풀이 과정
    where t' = applyTSubs tsubs t
sol ((t, TV i) : eqns) tsubs = sol ((TV i,t):eqns) tsubs -- 좌우반전

occurs :: Int -> Ty -> Bool
occurs i (TV j)      = i == j
occurs i Z           = False
occurs i (Arr ta tb) = occurs i ta || occurs i tb

applyTSubs :: TSubs -> Ty -> Ty
applyTSubs tsubs (TV i) = case tsubs !? i of Nothing -> TV i
                                             Just t  -> t
applyTSubs tsubs Z           = Z
applyTSubs tsubs (Arr ta tb) = Arr ta' tb'
                             where ta' = applyTSubs tsubs ta
                                   tb' = applyTSubs tsubs tb

insertTSubs :: Int -> Ty -> TSubs -> TSubs
insertTSubs i t tsubs = M.insert i t (M.map (applyTSubs i2t) tsubs)
                      where i2t = M.insert i t emptyTSubs

\noindent
\ref{chap:TyInf}장에서와 마찬가지 내용으로 아래의 `ev` 함수에서
`undefined`로 처리된 부분을 채워넣으면 된다.

인터프리터 함수 `ev`는 기존의 내용을 그대로 재사용하며
`eval'`은 이제 `infer`의 실행 결과만으로는 알맞은 타입이 있는지 판단할 수 없으므로
`solve`로 연립방정식의 해가 있는지도 확인한다. 이렇게 `infer`와 `solve`로 타입규칙에 알맞은
타입이 존재함이 확인되어 타입유추가 최종적으로 성공한 경우에만
아래의 인터프리터 함수 `ev`를 실행한다고 가정한다.

In [19]:
ev :: Env -> Expr -> Val  -- infer가 성공한 경우에만 실행함을 가정
ev env (Var x) = v    where Just v = lookup x env
ev env e@(Lam _ _ _) = Cl e env
ev env (Rec f t e@(Lam _ _ _)) = undefined -- 적절히 작성
ev env (App e1 e2) = ev ((x,v2):env1) e
    where Cl (Lam x _ e) env1 = ev env e1 
          v2                  = ev env e2 
ev _   (Lit n)     = Vi n
ev env (Add e1 e2) = Vi (n1 + n2)  where Vi n1 = ev env e1
                                         Vi n2 = ev env e2
ev env (If e e1 e0) = if n/=0 then ev env e1 else ev env e0
    where Vi n = ev env e

In [20]:
-- infer에 성공한 닫힌식에 대한 인터프리터
eval' e = [ev [] e | _ <- solve eqns]
  where (t,eqns) = infer' e

In [21]:
e4 = Rec "f" (Just $ Arr Z Z) .
         Lam "i" (Just Z) $ If i (i `Add` App f (i `Add` Lit (-1))) i
    where f = Var "f"
          i = Var "i"
e4 -- rec f (\i.if i t hen i + f(i + -1) else i)

\noindent
0부터 주어진 자연수 $i$까지의 총합을 구하는 재귀함수 `e4`를 위와 같이 작성하여
다음과 같이 실행해 보자.
`infer`, `solve`, `ev`함수의 `undefined` 부분을 잘 채워넣었다면
아래에 보이는 에러 없이 정상적으로 0부터 100까지의 합 5050이 계산될 것이다.

In [22]:
-- rec f (\i.if i then i + f(i + -1) else i) 100
eval' (e4 `App` Lit 100) -- 100부터 0까지의 총합

: 

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. TODO
1. TODO aaa TODO